In [1]:
!pip install -q datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|    

In [6]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('ChrisG19/Guatemala_info')

# Shuffle the dataset
dataset = dataset['train'].shuffle(seed=42)

# Split the dataset
train_dataset = dataset.select(range(10000))
test_dataset = dataset.select(range(10000, len(dataset)))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

# Apply the transformation
transformed_train_dataset = train_dataset.map(transform_conversation)
transformed_test_dataset = test_dataset.map(transform_conversation)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5375 [00:00<?, ? examples/s]

In [10]:
transformed_train_dataset.push_to_hub("quauhtlemallan-llama2-train-15k", "train")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ChrisG19/quauhtlemallan-llama2-train-15k/commit/0ccec226da56bf8c8acb3f0e2f512117cd7fbee9', commit_message='Upload dataset', commit_description='', oid='0ccec226da56bf8c8acb3f0e2f512117cd7fbee9', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
transformed_test_dataset.push_to_hub("quauhtlemallan-llama2-train-15k", "test")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/294 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ChrisG19/quauhtlemallan-llama2-train-15k/commit/c69883b2ea3c388cbc2c3abe6d27c5be16be5888', commit_message='Upload dataset', commit_description='', oid='c69883b2ea3c388cbc2c3abe6d27c5be16be5888', pr_url=None, pr_revision=None, pr_num=None)